### Curriculum Modeling: Resident Evaluation

In this notebook, we conduct the performance evaluation for the residentss trained with both interleaved and blocked (control) training schedules. 

Taining effect, both overall and by training condition (interleaved vs blocked), on accuracy, sensitivity, and specificity was assessed using mixed effects log binomial regression models. Accuracy, sensitivity, and specificity for each training condition were calculated using marginal means estimations and a comparison between pre- and post-test performance was assessed using Wald tests.

In this notebook, we will use the package lme4 to create our mixed effects regression models, and the package emmeans for marginal mean estimations.

To get started, we load the required packages:

In [2]:
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(boot)
require(lattice)
require(dplyr)

In [3]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 19044)

Matrix products: default

locale:
[1] LC_COLLATE=English_United States.1252 
[2] LC_CTYPE=English_United States.1252   
[3] LC_MONETARY=English_United States.1252
[4] LC_NUMERIC=C                          
[5] LC_TIME=English_United States.1252    

attached base packages:
[1] parallel  compiler  stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] forcats_0.4.0   stringr_1.4.0   dplyr_0.8.0.1   purrr_0.3.2    
 [5] readr_1.3.1     tidyr_0.8.3     tibble_2.1.1    ggplot2_3.1.1  
 [9] tidyverse_1.2.1 emmeans_1.3.4   magrittr_1.5    lattice_0.20-38
[13] boot_1.3-20     lme4_1.1-26     Matrix_1.2-17   reshape2_1.4.3 

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1       lubridate_1.7.4  mvtnorm_1.0-10   assertthat_0.2.1
 [5] digest_0.6.18    IRdisplay_0.7.0  R6_2.4.0         cellranger_1.1.0
 [9] plyr_1.8.4     

In [4]:
#load the radiology resident results
data <- read.csv('../all_participant_data.csv',na.strings=c(""," "))
data

ID,trainingLevel,stage,viewerInch,viewerPix,totalTimeMin,trialNum,trialOrder,caseID,caseType,...,LocX,LocY,clickX,clickY,clickTimeSecs,trialTimeSecs,response,responseType,correct,Condition
charlie,1,PreTest,12.166667,1168,41.66643,21,pTrial11,JPCLN059,Chest,...,565,1671,NA,NA,NA,194.100,Absent,Miss,No,NonBlocked
charlie,1,PreTest,12.166667,1168,41.66643,34,pTrial15,JPCLN051,Chest,...,524,1462,505,1468,178.328,180.578,Present,Hit,Yes,NonBlocked
hotel,NA,PostTest,8.791667,844,17.61012,32,pTrial28,P_01719,Mammo,...,1526,1139,1549,1182,175.875,176.732,Present,Hit,Yes,NonBlocked
hotel,1,PreTest,8.791667,844,26.34228,31,pTrial10,JPCLN072,Chest,...,401,1270,585,927,118.409,173.575,Present,Wrong Location,No,NonBlocked
yankee,NA,PostTest,15.718750,1509,19.81012,11,pTrial3,JPCNN008,Chest,...,NA,NA,NA,NA,NA,170.773,Absent,Correct Rejection,Yes,Blocked
charlie,1,PreTest,12.166667,1168,41.66643,9,pTrial14,JPCLN052,Chest,...,836,1167,851,1163,158.540,161.009,Present,Hit,Yes,NonBlocked
hotel,1,PreTest,8.791667,844,26.34228,10,pTrial20,JPCLN014,Chest,...,569,1331,556,1301,158.129,159.411,Present,Hit,Yes,NonBlocked
yankee,NA,PostTest,15.718750,1509,19.81012,29,pTrial14,JPCLN057,Chest,...,524,1343,537,763,150.129,154.190,Present,Wrong Location,No,Blocked
charlie,1,PreTest,12.166667,1168,41.66643,13,pTrial1,JPCNN005,Chest,...,NA,NA,NA,NA,NA,144.260,Absent,Correct Rejection,Yes,NonBlocked
charlie,1,PreTest,12.166667,1168,41.66643,35,pTrial13,JPCLN053,Chest,...,573,1528,874,946,136.768,139.709,Present,Wrong Location,No,NonBlocked


#### 1.0 Mixed Effects Log Binomial Model

In the following cells, we create a mixed effects log-binomial regression model of the form Correct ~ Time * Condition * Abnormality. The fold is added as a random effect. 

In [7]:
data$stage <- factor(data$stage, levels = c('PreTest', 'PostTest'))
data$abnormality <- factor(data$abnormality, levels = c('Yes', 'No'))

In [11]:
### Use full model for marginal means estimation though it was not the best model.
fm=as.formula (correct ~ stage+ Condition + abnormality + stage:abnormality+stage:Condition + Condition:abnormality + stage:Condition:abnormality + (1|ID))
fit=glmer(fm, family=binomial(log), control=glmerControl(optimizer="bobyqa"), data=data);
summary(fit)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( log )
Formula: correct ~ stage + Condition + abnormality + stage:abnormality +  
    stage:Condition + Condition:abnormality + stage:Condition:abnormality +  
    (1 | ID)
   Data: data
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
  1732.1   1778.4   -857.0   1714.1     1267 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.4697 -0.9356  0.6975  0.8998  1.3919 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 0.008525 0.09233 
Number of obs: 1276, groups:  ID, 16

Fixed effects:
                                                Estimate Std. Error z value
(Intercept)                                     -0.82830    0.08032 -10.312
stagePostTest                                    0.36781    0.08816   4.172
ConditionNonBlocked                             -0.14239    0.11955  -1.191
abnormalit

In [12]:
se <- sqrt(diag(vcov(fit)))

# table of estimates with 95% CI
tab <- cbind(Est = fixef(fit), LL = fixef(fit) - 1.96 * se, UL = fixef(fit) + 1.96 *
    se)

# calculate risk ratios by taking the exponential 
exp(tab)

,Est,LL,UL
(Intercept),0.4367902,0.3731635,0.5112656
stagePostTest,1.4445701,1.2153387,1.7170381
ConditionNonBlocked,0.8672832,0.6861113,1.0962946
abnormalityNo,1.3749637,1.0933843,1.7290584
stagePostTest:abnormalityNo,0.5606811,0.4016157,0.7827465
stagePostTest:ConditionNonBlocked,1.1111139,0.8594903,1.4364027
ConditionNonBlocked:abnormalityNo,0.9633080,0.6753750,1.3739957
stagePostTest:ConditionNonBlocked:abnormalityNo,1.2149278,0.7489379,1.9708572


In [14]:
# Calculate estimates and confidence intervals
ci <- confint(fit, method='Wald')[-1,]
est <- fixef(fit)
exp_est <- exp(est)
exp_ci <- exp(ci)

# Get p-values
p <- summary(fit)$coefficients[,4]

# Combine results into one dataframe
results <- data.frame(RR = exp_est,
                      ci_low = exp_ci[,1],
                      ci_high = exp_ci[,2],
                      p_value = p)

# Print results
results

,RR,ci_low,ci_high,p_value
(Intercept),0.4367902,0.3731646,0.5112641,6.223547e-25
stagePostTest,1.4445701,1.2153426,1.7170326,3.016656e-05
ConditionNonBlocked,0.8672832,0.6861142,1.0962899,2.336502e-01
abnormalityNo,1.3749637,1.0933889,1.7290512,6.456983e-03
stagePostTest:abnormalityNo,0.5606811,0.4016182,0.7827417,6.765996e-04
stagePostTest:ConditionNonBlocked,1.1111139,0.8594943,1.4363959,4.212590e-01
ConditionNonBlocked:abnormalityNo,0.9633080,0.6753794,1.3739868,8.365331e-01
stagePostTest:ConditionNonBlocked:abnormalityNo,1.2149278,0.7489445,1.9708397,4.302613e-01


In [16]:
library(emmeans)

# Calculate the EMMs and pairwise comparisons
emm.1 <- emmeans(fit, ~ stage)
pairwise <- as.data.frame(pairs(regrid(emm.1), transform = "log"))

# print the EMM - corresponding to Pre and Post-test accuracy in this case
summary(emm.1, type="response")

# Print the p.value
pairwise

NOTE: Results may be misleading due to involvement in interactions


stage,prob,SE,df,asymp.LCL,asymp.UCL
PreTest,0.4725418,0.02399422,Inf,0.4277784,0.5219894
PostTest,0.5656577,0.02717626,Inf,0.5148241,0.6215106


contrast,estimate,SE,df,z.ratio,p.value
PreTest - PostTest,-0.09311593,0.03193585,Inf,-2.915718,0.003548714


In [18]:
# Calculate the EMMs and pairwise comparisons
emm.1 <- emmeans(fit, ~ Condition * stage)
pairwise <- as.data.frame(pairs(regrid(emm.1), transform = "log"))

# print the EMM - corresponding to Pre and Post-test accuracy by training schedule
summary(emm.1, type="response")

# Print the p.value
pairwise

NOTE: Results may be misleading due to involvement in interactions


Condition,stage,prob,SE,df,asymp.LCL,asymp.UCL
Blocked,PreTest,0.5121751,0.03417838,Inf,0.4493826,0.5837417
NonBlocked,PreTest,0.4359754,0.03332726,Inf,0.3753130,0.5064428
Blocked,PostTest,0.5540068,0.03888580,Inf,0.4828021,0.6357129
NonBlocked,PostTest,0.5775537,0.03773274,Inf,0.5081380,0.6564522


contrast,estimate,SE,df,z.ratio,p.value
"Blocked,PreTest - NonBlocked,PreTest",0.07619973,0.04769884,Inf,1.5975177,0.379987663
"Blocked,PreTest - Blocked,PostTest",-0.04183164,0.04537390,Inf,-0.9219317,0.793121770
"Blocked,PreTest - NonBlocked,PostTest",-0.06537859,0.05079472,Inf,-1.2871137,0.571147763
"NonBlocked,PreTest - Blocked,PostTest",-0.11803137,0.05117649,Inf,-2.3063591,0.096539870
"NonBlocked,PreTest - NonBlocked,PostTest",-0.14157832,0.04461772,Inf,-3.1731414,0.008212939
"Blocked,PostTest - NonBlocked,PostTest",-0.02354695,0.05407176,Inf,-0.4354760,0.972359305


In [20]:
# Calculate the EMMs and pairwise comparisons
emm.1 <- emmeans(fit, ~ Condition * stage * abnormality)
pairwise <- as.data.frame(pairs(regrid(emm.1), transform = "log"))

# print the EMM - corresponding to Pre and Post-test sensitivity (abnormality YES) and specificity (abnormality NO)
# by training schedule (blocked vs non-blocked)
summary(emm.1, type="response")

# Print the p.value
pairwise

Condition,stage,abnormality,prob,SE,df,asymp.LCL,asymp.UCL
Blocked,PreTest,Yes,0.4367902,0.03508501,Inf,0.3731646,0.5112641
NonBlocked,PreTest,Yes,0.3788208,0.03354713,Inf,0.3184595,0.4506230
Blocked,PostTest,Yes,0.6309740,0.03716962,Inf,0.5621713,0.7081974
NonBlocked,PostTest,Yes,0.6080384,0.03715284,Inf,0.5394115,0.6853963
Blocked,PreTest,No,0.6005707,0.05788131,Inf,0.4971961,0.7254384
NonBlocked,PreTest,No,0.5017533,0.05811052,Inf,0.3998604,0.6296106
Blocked,PostTest,No,0.4864281,0.05781083,Inf,0.3853500,0.6140192
NonBlocked,PostTest,No,0.5485975,0.05797663,Inf,0.4459616,0.6748544


contrast,estimate,SE,df,z.ratio,p.value
"Blocked,PreTest,Yes - NonBlocked,PreTest,Yes",0.057969416,0.04854039,Inf,1.1942511,9.338423e-01
"Blocked,PreTest,Yes - Blocked,PostTest,Yes",-0.194183862,0.04494289,Inf,-4.3206807,4.135847e-04
"Blocked,PreTest,Yes - NonBlocked,PostTest,Yes",-0.171248190,0.05109523,Inf,-3.3515494,1.827882e-02
"Blocked,PreTest,Yes - Blocked,PreTest,No",-0.163780483,0.06354279,Inf,-2.5774834,1.640652e-01
"Blocked,PreTest,Yes - NonBlocked,PreTest,No",-0.064963071,0.06787983,Inf,-0.9570306,9.802002e-01
"Blocked,PreTest,Yes - Blocked,PostTest,No",-0.049637914,0.06421906,Inf,-0.7729467,9.944655e-01
"Blocked,PreTest,Yes - NonBlocked,PostTest,No",-0.111807271,0.06776236,Inf,-1.6499909,7.195960e-01
"NonBlocked,PreTest,Yes - Blocked,PostTest,Yes",-0.252153278,0.05000947,Inf,-5.0421107,1.269652e-05
"NonBlocked,PreTest,Yes - NonBlocked,PostTest,Yes",-0.229217606,0.04480401,Inf,-5.1160063,8.622155e-06
"NonBlocked,PreTest,Yes - Blocked,PreTest,No",-0.221749899,0.06682851,Inf,-3.3181932,2.041960e-02


#### 2.0 Comparison between radiology residents and the machine learners

Here we will make a mixed effects model that compares the residents to the machine learning algorithms.

In [3]:
hdata <- read.csv('../all_participant_data.csv',na.strings=c(""," "))
mdata <- read.csv('../reports/cnn_results.csv',na.strings=c(""," "))

In [4]:
#format columns
hdata$Correct = hdata$correct
hdata$Condition = tolower(hdata$Condition)
hdata$stage <- gsub("PreTest", "Pre", as.character(hdata$stage))
hdata$stage <- gsub("PostTest", "Post", as.character(hdata$stage))
hdata$Time <- hdata$stage
hdata$Abnormality <- hdata$abnormality
hdata = hdata %>% select("Condition", "Time", "Abnormality", "Correct", "ID")
mdata$ID = mdata$Fold
mdata = mdata %>% select("Condition", "Time", "Abnormality", "Correct", "ID")

# define learners
mdata$Learner = "Machine"
hdata$Learner = "Human"

In [5]:
# combine dataframes!
df <- rbind.data.frame(mdata,hdata)

In [6]:
df$Time <- factor(df$Time, levels = c('Pre', 'Post'))
df$Learner <- factor(df$Learner, levels = c("Machine", "Human"))
df$Abnormality <- factor(df$Abnormality, levels = c("Yes", "No"))
# Replace "Yes" with 1 and "No" with 0
df$Correct <- ifelse(df$Correct == "Yes", 1, 0)

In [7]:
# print the dataframe to ensure it was correctly formatted
df <- subset(df, Time == "Post")
df <- subset(df, Condition == "nonblocked")
df

,Condition,Time,Abnormality,Correct,ID,Learner
1,nonblocked,Post,Yes,1,fold_0,Machine
2,nonblocked,Post,Yes,1,fold_0,Machine
3,nonblocked,Post,No,1,fold_0,Machine
4,nonblocked,Post,No,0,fold_0,Machine
5,nonblocked,Post,Yes,0,fold_0,Machine
6,nonblocked,Post,Yes,1,fold_0,Machine
7,nonblocked,Post,No,0,fold_0,Machine
8,nonblocked,Post,Yes,1,fold_0,Machine
9,nonblocked,Post,Yes,1,fold_0,Machine
10,nonblocked,Post,No,0,fold_0,Machine


In [8]:
fm=as.formula (Correct ~ Learner + Abnormality * Learner + (1|ID))
m=glmer(fm, family=binomial(log), control=glmerControl(optimizer="bobyqa"), data=df, verbose=0);
summary(m)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( log )
Formula: Correct ~ Learner + Abnormality * Learner + (1 | ID)
   Data: df
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   694.3    715.6   -342.2    684.3      513 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.8746 -1.1585  0.6886  0.7874  0.9974 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 0.01115  0.1056  
Number of obs: 518, groups:  ID, 13

Fixed effects:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -0.38322    0.07375  -5.196 2.04e-07 ***
LearnerHuman               -0.11605    0.09775  -1.187    0.235    
AbnormalityNo              -0.17029    0.13218  -1.288    0.198    
LearnerHuman:AbnormalityNo  0.06822    0.17442   0.391    0.696    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation o

In [9]:
se <- sqrt(diag(vcov(m)))

# table of estimates with 95% CI
tab <- cbind(Est = fixef(m), LL = fixef(m) - 1.96 * se, UL = fixef(m) + 1.96 *
    se)

# calculate risk ratios by taking the exponential 
exp(tab)

,Est,LL,UL
(Intercept),0.6816630,0.5899153,0.7876799
LearnerHuman,0.8904319,0.7351814,1.0784671
AbnormalityNo,0.8434195,0.6509288,1.0928329
LearnerHuman:AbnormalityNo,1.0706025,0.7606107,1.5069335


In [10]:
# Calculate estimates and confidence intervals
ci <- confint(m, method='Wald')[-1,]
est <- fixef(m)
exp_est <- exp(est)
exp_ci <- exp(ci)

# Get p-values
p <- summary(m)$coefficients[,4]

# Combine results into one dataframe
results <- data.frame(RR = exp_est,
                      ci_low = exp_ci[,1],
                      ci_high = exp_ci[,2],
                      p_value = p)

# Print results
results

,RR,ci_low,ci_high,p_value
(Intercept),0.6816630,0.5899169,0.7876778,2.036539e-07
LearnerHuman,0.8904319,0.7351840,1.0784633,2.351478e-01
AbnormalityNo,0.8434195,0.6509319,1.0928277,1.976178e-01
LearnerHuman:AbnormalityNo,1.0706025,0.7606155,1.5069241,6.956914e-01


In [11]:
library(emmeans)

# Calculate the EMMs and pairwise comparisons
emm.1 <- emmeans(m, ~ Learner)
pairwise <- as.data.frame(pairs(regrid(emm.1), transform = "log"))

# print the EMM - corresponding to Pre and Post-test accuracy in this case
summary(emm.1, type="response")

# Print the p.value
pairwise

NOTE: Results may be misleading due to involvement in interactions


Learner,prob,SE,df,asymp.LCL,asymp.UCL
Machine,0.6260248,0.05058533,Inf,0.5343318,0.7334526
Human,0.5767749,0.03930072,Inf,0.5046689,0.6591833


contrast,estimate,SE,df,z.ratio,p.value
Machine - Human,0.04924985,0.06369623,Inf,0.7731989,0.4394047


In [12]:
emm.1 <- emmeans(m, ~ Learner * Abnormality)
pairwise <- as.data.frame(pairs(regrid(emm.1), transform = "log"))

# print the EMM - corresponding to Post-test sensitivity (abnormality YES) and specificity (abnormality NO)
# by training schedule (blocked vs non-blocked)
summary(emm.1, type="response")

# Print the p.value
pairwise

Learner,Abnormality,prob,SE,df,asymp.LCL,asymp.UCL
Machine,Yes,0.6816630,0.05027488,Inf,0.5899169,0.7876778
Human,Yes,0.6069745,0.03891752,Inf,0.5352957,0.6882514
Machine,No,0.5749279,0.07352389,Inf,0.4474645,0.7387001
Human,No,0.5480780,0.05912786,Inf,0.4436218,0.6771296


contrast,estimate,SE,df,z.ratio,p.value
"Machine,Yes - Human,Yes",0.07468851,0.06359222,Inf,1.1744912,0.6430044
"Machine,Yes - Machine,No",0.10673514,0.07898707,Inf,1.3512988,0.5301353
"Machine,Yes - Human,No",0.13358504,0.07762205,Inf,1.7209677,0.3126567
"Human,Yes - Machine,No",0.03204663,0.08261716,Inf,0.3878932,0.9801808
"Human,Yes - Human,No",0.05889654,0.06381140,Inf,0.9229783,0.7925564
"Machine,No - Human,No",0.02684990,0.09393305,Inf,0.2858409,0.9918639
